In [ ]:
import pyes
import elasticsearch

es_address='140.118.155.14:9200'
conn = pyes.es.ES(es_address)
bq = pyes.query.BoolQuery() 
#q = pyes.query.MatchAllQuery()

cve = pyes.query.TermQuery(field="text", value="cve")
vul = pyes.query.TermQuery(field="text", value="vulnerability")
exp = pyes.query.TermQuery(field="text", value="exploit")
back = pyes.query.TermQuery(field="text", value="backdoor")
apt = pyes.query.TermQuery(field="text", value="apt")
mal = pyes.query.TermQuery(field="text", value="malware")


ESR = pyes.ESRange(field="created_at_linux_timestamp", from_value="1388534400000", to_value="1420070400000", 
                   include_lower=True ,include_upper=False)
rq = pyes.query.RangeQuery(qrange=ESR)

bq.add_should(cve)
bq.add_should(vul)
bq.add_should(exp)
bq.add_should(back)
bq.add_should(apt)
bq.add_should(mal)
bq.add_must(rq)

result = conn.search(query=bq , indices='twitter2' , doc_types='tweet') 
len(result)

In [ ]:
from collections import defaultdict
twitters = set()
twitterPerMon = defaultdict(set)
tweetsPerMon = [0]*12
indicators = defaultdict(list)
twitterFollower = dict()

In [ ]:
import datetime
import re

haveUrl = 0
        
regex = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
for oneTweet in result:
    twitters.add(oneTweet['user']['id'])
    month = int(datetime.datetime.fromtimestamp(oneTweet['created_at_linux_timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S').split('-')[1])
    twitterPerMon[month - 1].add(oneTweet['user']['id'])
    tweetsPerMon[month - 1] += 1
    if regex.findall(oneTweet['text']):
        #print(regex.findall(oneTweet['text']))
        haveUrl += 1
    if oneTweet['user']['id'] not in indicators.iterkeys():
        indicators[oneTweet['user']['id']].append(1)
        indicators[oneTweet['user']['id']].append(0)
        indicators[oneTweet['user']['id']].append(0)
    else:
        indicators[oneTweet['user']['id']][0] += 1
        indicators[oneTweet['user']['id']][1] += oneTweet['retweet_count']
        indicators[oneTweet['user']['id']][2] += oneTweet['favorite_count']
    if oneTweet['user']['id'] not in twitterFollower.iterkeys():
        twitterFollower[oneTweet['user']['id']] = 0
    else:
        if twitterFollower[oneTweet['user']['id']] < oneTweet['user']['followers_count']:
            twitterFollower[oneTweet['user']['id']] = oneTweet['user']['followers_count']

noUrl = len(result) - haveUrl
print (len(twitters))
#print (twitterPerMon)
#print (tweetsPerMon)
#print (haveUrl, noUrl)
#print (indicators)
#len(indicators)
print(twitterFollower)

In [ ]:
from bokeh.charts import BoxPlot, Donut, Bar, output_notebook, show
from bokeh.charts.attributes import cat
sort=False
# best support is with data in a format that is table-like
data = {
    'tweets': tweetsPerMon,
    'label' : ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
    'twitter' : [len(twitter) for index, twitter in twitterPerMon.items() ]
}
print (data)

twitter = Bar(data, values='twitter', label=cat(columns='label', sort=False), title="Twitter Per Month")

tweet = Bar(data, values='tweets', label=cat(columns='label', sort=False), agg='mean', title="Tweets Per Month")


output_notebook()
show(twitter)
show(tweet)

In [ ]:
data = {
    'label' : ['haveUrl ' + str( float(haveUrl) / len(result) * 100) + "%", 'noUrl ' + str( float(noUrl) / len(result) * 100) + "%"],
    'value' : [haveUrl, noUrl]
}


pie = Donut(data, label='label', values='value', text_font_size='13pt', width=500)


output_notebook()

show(pie)

In [ ]:
bq2 = pyes.query.BoolQuery() 
bq2.add_should(cve)
bq2.add_must(rq)
justcve = conn.search(query=bq2 , indices='twitter2' , doc_types='tweet')
len(justcve)

In [ ]:
twitterMonthCVE = defaultdict(list)
for oneTweet in justcve:
    month = int(datetime.datetime.fromtimestamp(oneTweet['created_at_linux_timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S').split('-')[1])
    twitterMonthCVE[month - 1].append(oneTweet['user']['id'])
print (twitterMonthCVE)

In [ ]:
from collections import Counter
twitterMonthCountCVE = defaultdict(list)
for key, values in twitterMonthCVE.items():
    twitterMonthCountCVE['month'] += [key + 1] * len(dict(Counter(values)).values())
    twitterMonthCountCVE['account'] += dict(Counter(values)).values()
print (twitterMonthCountCVE)

In [ ]:
box = BoxPlot(twitterMonthCountCVE, values='account', label='month', title="Twitter CVE BoxPlot")

output_notebook()

show(box)

In [ ]:
twitterFollowers = sorted(twitterFollower.iteritems(), key=lambda (k,v):(v,k), reverse = True)[:10]
twitterFollowers = zip(*twitterFollowers)
data = {
    'label' : twitterFollowers[0],
    'flower' :  twitterFollowers[1]
}
print (data)

flower = Bar(data, values='flower', label=cat(columns='label', sort=False), title="Twitter_id Flower")



output_notebook()
show(flower)